In [1]:
%%bash
curl http://localhost:11434/api/chat -d '{
  "model": "qwen2.5-coder:32b",
  "messages": [
    {
      "role": "user",
      "content": "who wrote the book godfather?"
    }
  ],
  "stream": false,
  "options": {
    "temperature": 5,
    "num_predict": 75
    }
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   805  100   582  100   223     52     20  0:00:11  0:00:11 --:--:--   152


{"model":"qwen2.5-coder:32b","created_at":"2025-04-16T17:07:47.936525Z","message":{"role":"assistant","content":"The book \"The Godfather\" was written by Mario Puzo. It was published in 1969 and is one of the most famous works in modern literature, often considered a masterpiece of American crime fiction. The novel spawned a series that continues to influence popular culture to this day."},"done_reason":"stop","done":true,"total_duration":11049856500,"load_duration":667004375,"prompt_eval_count":36,"prompt_eval_duration":7386116500,"eval_count":61,"eval_duration":2995093625}

In [2]:
from itertools import islice
import ast
import os
import pandas as pd
import llm_run_functions as lf


def main():
    system_prompt_file='prompt_two_examples.txt'
    input_csv_path= '../sensitive/health_interview_proc_split.csv'
    batch_size=10

    i = 0

    output_csv_path = f"good/gemma3_27_temp0.csv"
    bad_csv_path = f"Bad_gemma3_27_temp0.csv"

    system_prompt = lf.read_system_prompt(system_prompt_file)
    input_df = pd.read_csv(input_csv_path)
    completed_ids = lf.read_completed_ids(output_csv_path)
    bad_ids = lf.read_completed_ids(bad_csv_path)


    input_df = input_df[~input_df['id'].isin(completed_ids)]
    input_df = input_df[~input_df['id'].isin(bad_ids)]
    print(f"Removed {len(completed_ids)} rows from input dataframe")
    print(f"Removed {len(bad_ids)} rows from input dataframe")
    print("Remaining rows: ", len(input_df))

    i = 0
    tot = len(input_df) // batch_size
    print(f"{tot}")
    for batch in lf.batch_iterator(input_df.iterrows(), batch_size):
        i += 1
        if i % 4 == 0:
            print(f"{i} : {tot}")
            # break
        batch_prompts = []
        batch_metadata = []
        results = []

        for index, row in batch:
            full_prompt = lf.generate_prompt(row, system_prompt)
            batch_prompts.append(full_prompt)
            this_metadata = {'id': row['id'], 'prompt': row['prompt'], 'answer': row['answer']}
            batch_metadata.append(this_metadata)
            res = lf.llama3(full_prompt)
            results.append(res)


        new_rows = []
        bad_rows = []
        for metadata, result in zip(batch_metadata, results):
            # print(result)
            if result is not None:
                # print(result)
                parsed_json = lf.parse_result(result)
                if parsed_json is not None:
                    metadata['json'] = parsed_json
                    new_rows.append(metadata)
                else:
                    # print("couldn't parse this: ", metadata[language_in])
                    metadata['result'] = result
                    bad_rows.append(metadata)

        if new_rows:
            print(f"Writing {len(new_rows)} rows to csv")
            # print(f"New ids completed: {[row['id'] for row in new_rows]}")
            new_df = pd.DataFrame(new_rows)
            if os.path.exists(output_csv_path):
                new_df.to_csv(output_csv_path, mode='a', header=False, index=False)
            else:
                new_df.to_csv(output_csv_path, index=False)

        if bad_rows:
            bad_df = pd.DataFrame(bad_rows)
            if os.path.exists(bad_csv_path):
                bad_df.to_csv(bad_csv_path, mode='a', header=False, index=False)
            else:
                bad_df.to_csv(bad_csv_path, index=False)

main()

/Users/ethan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Removed 9 rows from input dataframe
Removed 0 rows from input dataframe
Remaining rows:  568
56
JSON decoding error: Invalid \escape: line 4 column 79 (char 439)
Writing 9 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv
4 : 56
Writing 10 rows to csv
Writing 10 rows to csv
JSON decoding error: Invalid \escape: line 3 column 119 (char 378)
Writing 9 rows to csv
Writing 10 rows to csv
8 : 56
Writing 10 rows to csv
Writing 10 rows to csv
Writing 10 rows to csv


KeyboardInterrupt: 